In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
data_root = "/opt/ml/level1/mask_data/train"

In [3]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(data_root, 'train.csv'))
image_dir = os.path.join(data_root, 'images')

In [7]:
submission.age.max(), submission.age.min()

(60, 18)

In [8]:
submission['new_age'] = submission.age.map(lambda x : (x - submission.age.min()) / (submission.age.max() - submission.age.min()))

In [10]:
submission.age

0       45
1       52
2       54
3       58
4       59
        ..
2695    19
2696    19
2697    19
2698    20
2699    19
Name: age, Length: 2700, dtype: int64

In [9]:
submission.new_age

0       0.642857
1       0.809524
2       0.857143
3       0.952381
4       0.976190
          ...   
2695    0.023810
2696    0.023810
2697    0.023810
2698    0.047619
2699    0.023810
Name: new_age, Length: 2700, dtype: float64

In [7]:
submission.age.describe()

count    2700.000000
mean       37.708148
std        16.985904
min        18.000000
25%        20.000000
50%        36.000000
75%        55.000000
max        60.000000
Name: age, dtype: float64

In [16]:
import glob
image_paths = glob.glob(os.path.join(image_dir, '*', '*'))

In [40]:
def path_to_label(paths):
    labels = []
    for path in paths:
        label_num = []
        label = path.split(os.path.sep)[-2:]
        if 'incorrect' in label[1]:
            label_num.append(2)
        elif 'mask' in label[1]:
            label_num.append(1)
        else: label_num.append(0)

        gender_age = label[0].split('_')
        gender = 0 if gender_age[1] == 'female' else 1
        age = int(gender_age[-1])
        label_num.extend([gender, age])
        labels.append(label_num)
    return labels

In [41]:
labels = path_to_label(image_paths)    

In [42]:
len(labels)

18900

In [43]:
labels[:20]

[[1, 0, 60],
 [0, 0, 60],
 [1, 0, 60],
 [1, 0, 60],
 [1, 0, 60],
 [1, 0, 60],
 [2, 0, 60],
 [1, 0, 55],
 [0, 0, 55],
 [1, 0, 55],
 [1, 0, 55],
 [1, 0, 55],
 [1, 0, 55],
 [2, 0, 55],
 [1, 1, 19],
 [1, 1, 19],
 [2, 1, 19],
 [0, 1, 19],
 [1, 1, 19],
 [1, 1, 19]]